In [1]:
import itertools
import pandas as pd
import numpy as np
import toytree
import toyplot
import arviz as az
import pymc3 as pm

### Tree and dataframe setup

In [2]:
# generate a random tree
NSPECIES = 80
TREE = toytree.rtree.bdtree(
    ntips=NSPECIES,
    seed=666,
).mod.node_scale_root_height(1.0)

# node idxs that delimit several distinct clades on this tree
CLADES = [152, 153, 154, 155]

# draw and color the four major clades
TREE.draw(
    layout='d', 
    width=500,
    tip_labels=False,
    edge_colors=TREE.get_edge_values_mapped({
        j: toytree.colors[i] for i, j in enumerate(CLADES)
    }),
    scalebar=True,
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t2a3fd5252204487591ca79242ca8000d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0.0 0.5 1.0

In [4]:
# make group index (gidx)
crown_dict = {i: TREE.get_tip_labels(i) for i in CLADES}
gidx = np.zeros(TREE.ntips, dtype=int)
for tidx, tip in enumerate(TREE.get_tip_labels()):
    for cidx, clade in enumerate(crown_dict):
        if tip in crown_dict[clade]:
            gidx[tidx] = cidx
gidx

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [3]:
# True param values
𝛼_mean = 0.05
𝛼_std = 0.02
𝛽_mean = 3.0
𝛽_std = 0.2
𝜓_mean = 0.0
𝜓_std = 0.33

# 4 different clade effects on rate of RI (used for partial-pooling data)
𝜓_0_mean = 1.0
𝜓_0_std = 0.1
𝜓_1_mean = 0.5
𝜓_1_std = 0.05
𝜓_2_mean = -0.5
𝜓_2_std = 0.05
𝜓_3_mean = -1.0
𝜓_3_std = 0.1

In [5]:
# species dataframe
SPECIES_DATA = pd.DataFrame({
    "gidx": gidx,
    "b": np.random.normal(𝛽_mean, 𝛽_std, TREE.ntips),
    "psi": np.random.normal(𝜓_mean, 𝜓_std, TREE.ntips),
    "psi_x": np.concatenate([
        np.random.normal(𝜓_0_mean, 𝜓_0_std, len(gidx[gidx == 0])),
        np.random.normal(𝜓_1_mean, 𝜓_1_std, len(gidx[gidx == 1])),
        np.random.normal(𝜓_2_mean, 𝜓_2_std, len(gidx[gidx == 2])),
        np.random.normal(𝜓_3_mean, 𝜓_3_std, len(gidx[gidx == 3])),
    ]),
})
SPECIES_DATA.head()

,gidx,b,psi,psi_x
0,0,2.570468,-0.236524,0.997529
1,0,3.292698,-0.624017,0.863576
2,0,2.936227,-0.612795,0.956139
3,0,2.746916,-0.234236,1.115301
4,0,2.767121,0.031270,0.981499


### Generate crossing data

In [6]:
def get_dist(tree, idx0, idx1):
    "returns the genetic distance between two nodes on a tree"
    dist = tree.treenode.get_distance(
        tree.idx_dict[idx0], 
        tree.idx_dict[idx1],
    )
    return dist

# get all combinations of two sampled taxa
a, b = zip(*itertools.combinations(range(NSPECIES), 2))

# organize into DF and get genetic distance between pairs
DATA = pd.DataFrame({
    "sidx0": a,
    "sidx1": b,
    "dist": [(get_dist(TREE, i, j) / 2) for (i, j) in zip(a, b)],
})

DATA['b'] = np.random.normal(𝛽_mean, 𝛽_std, DATA.shape[0])
DATA['velo'] = (
    DATA['b']
    + SPECIES_DATA['psi'][DATA.sidx0].values
    + SPECIES_DATA['psi'][DATA.sidx1].values
)
DATA['velo_x'] = (
    DATA['b']
    + SPECIES_DATA['psi_x'][DATA.sidx0].values
    + SPECIES_DATA['psi_x'][DATA.sidx1].values
)
DATA['intercept'] = np.random.normal(𝛼_mean, 𝛼_std, DATA.shape[0])
# DATA['error'] = np.random.normal(0.0, 𝜎_std, DATA.shape[0])

# get logits
DATA['logit_b'] = (
    1 / (1 + np.exp(-(DATA.b * DATA.dist ** 2 + DATA.intercept * DATA.dist)))
)
DATA['logit'] = (
    1 / (1 + np.exp(-(DATA.velo * DATA.dist ** 2 + DATA.intercept * DATA.dist)))
)
DATA['logit_x'] = (
    1 / (1 + np.exp(-(DATA.velo_x * DATA.dist ** 2 + DATA.intercept * DATA.dist)))
)

# get RI estimates
DATA['RI_pooled'] = np.random.binomial(n=1, p=DATA.logit_b / DATA.logit_b.max())
DATA['RI_unpooled'] = np.random.binomial(n=1, p=DATA.logit / DATA.logit.max())
DATA['RI_partpooled'] = np.random.binomial(n=1, p=DATA.logit_x / DATA.logit_x.max())

DATA.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,logit_b,logit,logit_x,RI_pooled,RI_unpooled,RI_partpooled
0,0,1,0.073376,2.881579,2.021039,4.742684,0.023752,0.504314,0.503156,0.506819,0,0,0
1,0,2,0.089748,2.638508,1.789189,4.592176,0.074466,0.506983,0.505273,0.510916,1,1,0
2,0,3,0.129770,3.113867,2.643107,5.226697,0.055129,0.514894,0.512913,0.523775,0,1,0
3,0,4,0.144542,3.059197,2.853944,5.038226,0.059521,0.518121,0.517051,0.528435,0,0,1
4,0,5,0.144542,2.620887,2.866354,4.699434,0.059066,0.515818,0.517099,0.526655,0,0,0


In [7]:
NSAMPLES = 1000
SAMPLE = DATA.sample(NSAMPLES).copy().reset_index(drop=True)
SAMPLE.head()

,sidx0,sidx1,dist,b,velo,velo_x,intercept,logit_b,logit,logit_x,RI_pooled,RI_unpooled,RI_partpooled
0,60,63,0.578689,2.921440,2.625589,1.913308,0.044766,0.731892,0.712010,0.660750,1,1,1
1,46,74,0.592720,3.287330,3.246154,1.886728,0.043472,0.765067,0.762457,0.665657,0,0,0
2,23,30,0.438896,2.615066,2.010742,4.803178,0.057368,0.629231,0.601689,0.721198,1,0,1
3,61,71,0.592720,3.160457,2.911351,1.778620,0.020785,0.754479,0.737909,0.654116,1,1,1
4,23,34,0.590047,3.069562,2.832857,4.990309,0.039344,0.748739,0.732920,0.853287,1,1,1


### Visualize data

In [8]:
def logit_plot(dist, logit, RI):
    canvas = toyplot.Canvas(width=500, height=250)
    ax0 = canvas.cartesian(
        label="pooled data (function)",
        xlabel="Genetic dist.",
        ylabel="Logit function",
        grid=(1, 2, 0),
    )
    ax1 = canvas.cartesian(
        label="pooled data (observation)",
        xlabel="Genetic dist.",
        ylabel="RI",
        grid=(1, 2, 1),
    )

    # points are jittered on x-axis for visibility
    ax0.scatterplot(
        dist,
        logit,
        size=5,
        opacity=0.33,
        color=toyplot.color.Palette()[0],
    );
    ax1.scatterplot(
        dist,
        RI,
        size=10,
        opacity=0.2,
        marker="|",
        mstyle={
            "stroke": toyplot.color.Palette()[1],
            "stroke-width": 3,
        },
    );
    return canvas, (ax0, ax1)

In [11]:
logit_plot(SAMPLE.dist, SAMPLE.logit_b, SAMPLE.RI_pooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t36218f2d92344c96af03dc884670ad5a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:rgb(40%,76.1%,64.7%

In [10]:
logit_plot(SAMPLE.dist, SAMPLE.logit, SAMPLE.RI_unpooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t6ee68bbb5ed54959ab2c4533681077e6" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:rgb(40%,76.1%,64.7%);st

In [12]:
logit_plot(SAMPLE.dist, SAMPLE.logit_x, SAMPLE.RI_partpooled);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="te5b584e48c364c4bb03d6df8bfd53269" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:rgb(40%,76.1%,64.7%);stroke-opacit

### Define models

In [13]:
def pooled_logistic(x, y, **kwargs):
    
    # define model
    with pm.Model() as model:  

        # parameters and error
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        
        # link function
        effect = (𝛽 * x ** 2) + 𝛼 * x
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood
        y = pm.Bernoulli("y", p=logit, observed=y)
        
        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [14]:
def unpooled_logistic(x, y, idx0, idx1, **kwargs):
    
    # define model
    with pm.Model() as model:
        
        # indexers
        sidx0 = pm.Data("spp_idx0", idx0.values)
        sidx1 = pm.Data("spp_idx1", idx1.values)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=1)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=1)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean + 𝜓_std * 𝜓_offset)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        
        # link function
        effect = ((𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * x ** 2) + 𝛼 * x
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood
        y = pm.Bernoulli("y", p=logit, observed=y)
        
        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]
    
        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

In [15]:
def partpooled_logistic(x, y, idx0, idx1, gidx, **kwargs):
    
    # define model
    with pm.Model() as model:
        
        # indexers
        sidx0 = pm.Data("spp_idx0", idx0)
        sidx1 = pm.Data("spp_idx1", idx1)
        gidx = pm.Data("gidx", gidx)

        # parameters and error
        𝜓_mean = pm.Normal('𝜓_mean', mu=0., sigma=5., shape=4)
        𝜓_std = pm.HalfNormal('𝜓_std', 5., shape=4)
        𝜓_offset = pm.Normal('𝜓_offset', mu=0, sigma=1., shape=TREE.ntips)
        𝜓 = pm.Deterministic('𝜓', 𝜓_mean[gidx] + 𝜓_std[gidx] * 𝜓_offset)
        𝛽 = pm.Normal('𝛽', mu=0., sigma=10., shape=1)
        𝛼 = pm.Normal('𝛼', mu=0., sigma=10., shape=1)
        
        # linear model prediction
        effect = ((𝛽 + 𝜓[sidx0] + 𝜓[sidx1]) * x ** 2) + 𝛼 * x
        logit = pm.Deterministic("logit", pm.invlogit(effect))
        
        # data likelihood (normal distributed errors)
        y = pm.Bernoulli("y", p=logit, observed=y)

        # sample posterior, skip burnin
        trace = pm.sample(**kwargs)[1000:]

        # show summary table
        stats = pm.summary(trace)
        
    # organize results
    result_dict = {
        'model': model, 
        'trace': trace,
        'stats': stats,
    }
    return result_dict

### Functions to plot results

In [16]:
def toytrace(trace, var_names, titles):
    """
    Plot posterior trace with toyplot
    """
    nvars = len(var_names)
    
    # setup canvase
    canvas = toyplot.Canvas(width=500, height=200 * nvars)
    
    # store axes
    axes = []
    
    # iter over params
    for pidx, param in enumerate(var_names):
        
        # get param posterior
        posterior = trace.get_values(param)
        
        # setup axes 
        ax = canvas.cartesian(grid=(nvars, 1, pidx))
        ax.y.show = False
        ax.x.spine.style = {"stroke-width": 1.5}
        ax.x.ticks.labels.style = {"font-size": "12px"}
        ax.x.ticks.show = True
        ax.x.label.text = f"param='{titles[pidx]}'"        
        
        # iterate over shape of param
        for idx in range(posterior.shape[1]):
            mags, bins = np.histogram(posterior[:, idx], bins=100)
            ax.plot(bins[1:], mags, stroke_width=2, opacity=0.6)
        axes.append(ax)
    return canvas, axes

In [17]:
import scipy.stats as stats

def draw_velocity_dists(trace, baseline=0.15):
    """
    Draw the clade velocities as gaussians
    """
    canvas = toyplot.Canvas(width=350, height=300)
    axes = canvas.cartesian(xlabel="Relative velocity of reproductive isolation")
    marks = []
    base = 0
    for i in range(trace['𝜓_mean'].shape[1]):
        
        loc = trace['𝜓_mean'][:, i].mean()
        scale = trace['𝜓_std'][:, i].mean()
        interval = stats.norm.interval(0.995, loc, scale)
        points = np.linspace(interval[0], interval[1], 100)
        mark = axes.fill(
            points, 
            stats.norm.pdf(points, loc=loc, scale=scale), 
            style={
                "fill-opacity": 0.45,
                "stroke": 'black',
                "stroke-opacity": 1.0,
                "stroke-width": 1,
            },
            baseline=np.repeat(base, 100),
        )
        marks.append(mark)
        axes.hlines(base, style={"stroke-dasharray": "5,5", 'stroke-width': 1})
        base += baseline
        
    axes.y.show = False
    axes.x.ticks.locator = toyplot.locator.Extended(only_inside=True)
    axes.x.ticks.show = True
    return canvas, axes, marks

In [ ]:
# MCMC sampler kwargs
sample_kwargs = dict(
    tune=10000,
    draws=10000,
    target_accept=0.99,
    return_inferencedata=False,
    progressbar=True,
)

### Run three datasets under pooled model

In [18]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_pooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

In [19]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_unpooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

INFO (theano.gof.compilelock): Waiting for existing lock by process '57440' (I am process '59036')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/henry/.theano/compiledir_Linux-4.15--generic-x86_64-with-debian-buster-sid-x86_64-3.7.9-64/lock_dir
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 220 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# pooled model
pooled_model_partpooled_data = pooled_logistic(*model_args[:2], **sample_kwargs)

### Run three datasets under unpooled model

In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_pooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

In [21]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_unpooled_data  = unpooled_logistic(*model_args[:4], **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛽, 𝛼, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 7198 seconds.


In [ ]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
unpooled_model_partpooled_data = unpooled_logistic(*model_args[:4], **sample_kwargs)

### Run three datasets under partpooled model

In [64]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_pooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_pooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 3847 seconds.


In [65]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_unpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_unpooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 3896 seconds.
The number of effective samples is smaller than 25% for some parameters.


In [66]:
# model input
model_args = [
    SAMPLE.dist,
    SAMPLE.RI_partpooled,
    SAMPLE.sidx0,
    SAMPLE.sidx1,
    gidx
]

# unpooled model
partpooled_model_partpooled_data = partpooled_logistic(*model_args, **sample_kwargs)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [𝛼, 𝛽, 𝜓_offset, 𝜓_std, 𝜓_mean]


Sampling 4 chains for 10_000 tune and 10_000 draw iterations (40_000 + 40_000 draws total) took 4034 seconds.


In [67]:
partpooled_model_partpooled_data['stats']

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
𝜓_mean[0],1.471,2.264,-2.749,5.734,0.020,0.014,12915.0,12915.0,12917.0,20148.0,1.0
𝜓_mean[1],1.274,2.271,-3.041,5.485,0.020,0.014,13186.0,13186.0,13187.0,20080.0,1.0
𝜓_mean[2],0.164,2.262,-4.049,4.450,0.020,0.014,12886.0,12886.0,12890.0,20505.0,1.0
𝜓_mean[3],-0.912,2.270,-5.122,3.396,0.020,0.014,13087.0,13087.0,13095.0,20444.0,1.0
𝜓_offset[0],0.076,0.901,-1.588,1.809,0.003,0.006,73466.0,12920.0,73567.0,24930.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
logit[995],0.959,0.047,0.880,1.000,0.000,0.000,30393.0,30393.0,33273.0,30447.0,1.0
logit[996],0.987,0.015,0.963,1.000,0.000,0.000,34795.0,34795.0,36182.0,28975.0,1.0
logit[997],0.972,0.034,0.920,1.000,0.000,0.000,34300.0,34300.0,46660.0,29555.0,1.0
logit[998],0.589,0.036,0.520,0.659,0.000,0.000,68052.0,67443.0,68113.0,30515.0,1.0


In [68]:
toytrace(partpooled_model_partpooled_data['trace'], ['𝜓_mean', '𝜓_offset', '𝜓'], ['psi-mean', 'psi-offset', 'psi-spp']);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="tad3a789f4a93485a9859121588459013" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 600.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 102.16332883073949 149.9210110584518 L 105.6768305597219 149.9210110584518 L 109.19033228870435 149.84202211690362 L 112.70383401768677 149.76303317535545 L 116.2173357466692 149.84202211690362 L 119.73083747565164 149.76303317535545 L 123.24433920463406 149.68404423380727 L 126.7578409336165 149.6050552922591 L 130.27134266259893 149.36808846761454 L 133.78484439158137 149.5260663507109 L 137.2983461205638 148.65718799368088 L 140.8118478495462 148.81516587677726 L 144.32534957852866 148.34123222748815 L 147.83885130751105 148.49921011058453 L 151.3523530364935 147.5513428120063 L 154.86585476547594 146.91943127962082 L 158.37935649445836 146.99842022116903 L 161.89285822344078 145.10268562401265 L 165.40635995242323 146.12954186413901 L 168.91986168140565 142.9699842022117 L 172.4333634103881 143.28593996840442 L 175.94686513937052 139.88941548183254 L 179.46036686835293 139.652448657188 L 182.97386859733535 137.51974723538703 L 186.48737032631777 133.72827804107425 L 190.00087205530022 131.43759873617694 L 193.51437378428264 128.3570300157978 L 197.0278755132651 125.9873617693523 L 200.54137724224748 122.43285939968405 L 204.05487897122995 117.14060031595577 L 207.5683807002124 111.7693522906793 L 211.08188242919482 110.74249605055294 L 214.5953841581772 106.39810426540284 L 218.10888588715966 100.78988941548184 L 221.6223876161421 93.99684044233808 L 225.13588934512453 90.52132701421802 L 228.64939107410697 86.57187993680886 L 232.1628928030894 86.01895734597157 L 235.6763945320718 79.77883096366509 L 239.18989626105426 73.30173775671406 L 242.7033979900367 68.08846761453397 L 246.21689971901912 77.3301737756714 L 249.73040144800154 66.11374407582937 L 253.24390317698396 63.03317535545024 L 256.7574049059664 56.477093206951025 L 260.2709066349488 53.080568720379155 L 263.7844083639313 55.13428120063191 L 267.29791009291364 50.0 L 270.8114118218961 52.52764612954186 L 274.3249135508786 60.18957345971563 L 277.838415279861 61.137440758293835 L 281.3519170088434 62.796208530805686 L 284.86541873782585 62.480252764612956 L 288.37892046680827 63.27014218009479 L 291.8924221957907 70.53712480252764 L 295.4059239247731 75.59241706161137 L 298.9194256537555 81.35860979462876 L 302.432927382738 89.49447077409164 L 305.94642911172036 89.88941548183254 L 309.45993084070284 93.12796208530807 L 312.97343256968526 97.31437598736176 L 316.4869342986677 102.44865718799367 L 320.0004360276501 109.71563981042652 L 323.5139377566326 111.13744075829385 L 327.027439485615 116.82464454976304 L 330.5409412145974 118.32543443917851 L 334.05444294357983 122.51184834123224 L 337.56794467256225 127.80410742496052 L 341.0814464015447 126.93522906793049 L 344.59494813052714 132.5434439178515 L 348.10844985950956 135.3870458135861 L 351.62195158849204 137.36176935229065 L 355.13545331747434 138.62559241706163 L 358.6489550464568 140.36334913112162 L 362.1624567754393 140.9952606635071 L 365.6759585044217 144.391785150079 L 369.1894602334042 145.02369668246445 L 372.70296196238655 145.7345971563981 L 376.21646369136903 147.0774091627172 L 379.7299654203514 147.31437598736179 L 383.2434671493338 147.86729857819904 L 386.7569688783163 148.65718799368088 L 390.2704706072987 148.1832543443918 L 393.7839723362812 148.9731437598736 L 397.2974740652636 149.21011058451816 L 400.81097579424596 149.28909952606634 L 404.32447752322844 149.5260663507109 L 407.8379792522108 149.76303317535545 L 411.3514809811932 149.60

In [69]:
# show plot of TRUE vs. ESTIMATED rates
c, a, m = toyplot.scatterplot(
    partpooled_model_partpooled_data['trace']['𝜓'].mean(axis=0),         # estimated
    SPECIES_DATA['psi_x'],                             # true
    width=400,
    height=250,
    xlabel="ESTIMATED species velocity",
    ylabel="TRUE species velocity",
    color=[toyplot.color.Palette()[i] for i in SPECIES_DATA.gidx],
);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t6a7871ae1ae544bbb337832f083e559f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 250.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> -1 0 1 2 ESTIMATED species velocity -1 0 1 TRUE species velocity

In [70]:
draw_velocity_dists(partpooled_model_partpooled_data['trace'], baseline = 1.2);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tb3f2a2f51cd14cce88a3e7c9cd773337" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <polygon points="126.60337766027277,250.0 128.35485869400742,250.0 130.106339727742,250.0 131.85782076147663,250.0 133.60930179521125,250.0 135.36078282894587,250.0 137.1122638626805,250.0 138.8637448964151,250.0 140.61522593014973,250.0 142.36670696388435,250.0 144.11818799761897,250.0 145.8696690313536,250.0 147.62115006508822,250.0 149.37263109882284,250.0 151.12411213255746,250.0 152.87559316629205,250.0 154.62707420002667,250.0 156.37855523376132,250.0 158.1300362674959,250.0 159.88151730123053,250.0 161.63299833496515,250.0 163.38447936869974,250.0 165.1359604024344,250.0 166.887441436169,250.0 168.6389224699036,250.0 170.39040350363825,250.0 172.14188453737287,250.0 173.89336557110747,250.0 175.6448466048421,250.0 177.39632763857674,250.0 179.14780867231133,250.0 180.89928970604595,250.0 182.65077073978057,250.0 184.40225177351516,250.0 186.15373280724978,250.0 187.90521384098443,250.0 189.65669487471905,250.0 191.40817590845364,250.0 193.15965694218826,250.0 194.91113797592288,250.0 196.66261900965753,250.0 198.41410004339215,250.0 200.16558107712675,250.0 201.91706211086137,250.0 203.668543144596,250.0 205.42002417833058,250.0 207.1715052120652,250.0 208.92298624579985,250.0 210.67446727953444,250.0 212.42594831326906,250.0 214.17742934700368,250.0 215.92891038073833,250.0 217.68039141447292,250.0 219.43187244820754,250.0 221.18335348194216,250.0 222.93483451567676,250.0 224.6863155494114,250.0 226.43779658314602,250.0 228.18927761688062,250.0 229.94075865061524,250.0 231.69223968434986,250.0 233.44372071808445,250.0 235.19520175181913,250.0 236.94668278555372,250.0 238.69816381928834,250.0 240.44964485302296,250.0 242.20112588675758,250.0 243.95260692049217,250.0 245.70408795422682,250.0 247.45556898796144,250.0 249.20705002169603,250.0 250.95853105543068,250.0 252.71001208916528,250.0 254.46149312289987,250.0 256.2129741566345,250.0 257.9644551903691,250.0 259.7159362241037,250.0 261.4674172578384,250.0 263.218898291573,250.0 264.9703793253076,250.0 266.72186035904224,250.0 268.4733413927769,250.0 270.2248224265115,250.0 271.97630346024613,250.0 273.72778449398066,250.0 275.47926552771526,250.0 277.23074656144996,250.0 278.98222759518455,250.0 280.7337086289192,250.0 282.4851896626538,250.0 284.2366706963884,250.0 285.9881517301231,250.0 287.7396327638577,250.0 289.4911137975923,250.0 291.24259483132687,250.0 292.99407586506146,250.0 294.7455568987961,250.0 296.49703793253076,250.0 298.24851896626535,250.0 300.0,250.0 300.0,249.5422593603125 298.24851896626535,249.46413866325133 296.49703793253076,249.3746994895168 294.7455568987961,249.2726749382627 292.99407586506146,249.15672013476686 291.24259483132687,249.0254181513105 289.4911137975923,248.87728812750206 287.7396327638577,248.710795771837 285.9881517301231,248.52436639738727 284.2366706963884,248.31640060691376 282.4851896626538,248.08529269622645 280.7337086289192,247.82945178937763 278.98222759518455,247.54732565569486 277.23074656144996,247.23742708751104 275.47926552771526,246.89836263986172 273.72778449398066,246.52886345091548 271.97630346024613,246.12781777635513 270.2248224265115,245.69430478457065 268.4733413927769,245.22762907489283 266.72186035904224,244.72735530098265 264.9703793253076,244.19334220885816 263.218898291573,243.62577533695134 261.4674172578384,243.02519757707893 259.7159362241037,242.39253676320305 257.9644551903691,241.7291294420083 256.2129741566345,241.03673998792812 254.46149312289987,240.31757

### Assess model fit

In [30]:
def rmse(predictions, targets):
    differences = predictions - targets                       #the DIFFERENCEs.
    differences_squared = differences ** 2                    #the SQUAREs of ^
    mean_of_differences_squared = differences_squared.mean()  #the MEAN of ^
    rmse_val = np.sqrt(mean_of_differences_squared)           #ROOT of ^
    return rmse_val

In [31]:
def aicm(mean, variance):
    return 2*mean - 2*variance

In [32]:
rmse(unpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi'])

0.9292188416833755

In [33]:
rmse(partpooled_sub['trace']['𝜓'].mean(axis=0), SPECIES_DATA['psi_x'])

0.6797919539022527

In [34]:
aicm(unpooled_sub['trace']['𝜓'].mean(), unpooled_sub['trace']['𝜓'].var())

-23.831186286890336

In [35]:
# Raftery et al. (2007) holds that this value is better.  R+M disagree?
aicm(partpooled_sub['trace']['𝜓'].mean(), partpooled_sub['trace']['𝜓'].var())

-11.997795212343398

In [71]:
az_pooled_model_pooled_data = az.from_pymc3(trace = pooled_model_pooled_data['trace'], 
                                            model = pooled_model_pooled_data['model'])
az_pooled_model_unpooled_data = az.from_pymc3(trace = pooled_model_unpooled_data['trace'], 
                                            model = pooled_model_unpooled_data['model'])
az_pooled_model_partpooled_data = az.from_pymc3(trace = pooled_model_partpooled_data['trace'], 
                                            model = pooled_model_partpooled_data['model'])
az_unpooled_model_pooled_data = az.from_pymc3(trace = unpooled_model_pooled_data['trace'], 
                                            model = unpooled_model_pooled_data['model'])
az_unpooled_model_unpooled_data = az.from_pymc3(trace = unpooled_model_unpooled_data['trace'], 
                                            model = unpooled_model_unpooled_data['model'])
az_unpooled_model_partpooled_data = az.from_pymc3(trace = unpooled_model_partpooled_data['trace'], 
                                            model = unpooled_model_partpooled_data['model'])
az_partpooled_model_pooled_data = az.from_pymc3(trace = partpooled_model_pooled_data['trace'], 
                                            model = partpooled_model_pooled_data['model'])
az_partpooled_model_unpooled_data = az.from_pymc3(trace = partpooled_model_unpooled_data['trace'], 
                                            model = partpooled_model_unpooled_data['model'])
az_partpooled_model_partpooled_data = az.from_pymc3(trace = partpooled_model_partpooled_data['trace'], 
                                            model = partpooled_model_partpooled_data['model'])

In [72]:
az.compare({"pooled_model_pooled_data": az_pooled_model_pooled_data,
            "pooled_model_unpooled_data": az_pooled_model_unpooled_data, 
            "pooled_model_partpooled_data": az_pooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
pooled_model_pooled_data,0,-317.914,1.84283,0,0.938013,17.4455,0,False,log
pooled_model_partpooled_data,1,-352.349,2.12846,34.4356,0.0619875,17.8978,22.3327,False,log
pooled_model_unpooled_data,2,-405.033,2.02673,87.119,3.64907e-15,18.5206,22.1702,False,log


In [73]:
az.compare({"unpooled_model_pooled_data": az_unpooled_model_pooled_data,
            "unpooled_model_unpooled_data": az_unpooled_model_unpooled_data, 
            "unpooled_model_partpooled_data": az_unpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
unpooled_model_pooled_data,0,-318.459,7.98969,0,0.940292,17.5642,0,False,log
unpooled_model_partpooled_data,1,-352.555,15.0592,34.0955,0.0597076,18.0702,22.5237,False,log
unpooled_model_unpooled_data,2,-405.831,10.4153,87.3717,1.99245e-12,18.1106,22.2593,False,log


In [74]:
az.compare({"partpooled_model_pooled_data": az_partpooled_model_pooled_data,
            "partpooled_model_unpooled_data": az_partpooled_model_unpooled_data, 
            "partpooled_model_partpooled_data": az_partpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
partpooled_model_pooled_data,0,-321.778,20.475,0,0.852327,18.0864,0,False,log
partpooled_model_partpooled_data,1,-347.201,22.2082,25.4233,0.147673,18.3211,23.1761,False,log
partpooled_model_unpooled_data,2,-409.653,21.3569,87.8759,2.75963e-13,18.209,22.8055,False,log


In [75]:
az.compare({"pooled_model_pooled_data": az_pooled_model_pooled_data,
            "unpooled_model_unpooled_data": az_unpooled_model_unpooled_data, 
            "partpooled_model_partpooled_data": az_partpooled_model_partpooled_data})

/home/henry/miniconda3/envs/py37/lib/python3.7/site-packages/arviz/stats/stats.py:150: UserWarning: 
The scale is now log by default. Use 'scale' argument or 'stats.ic_scale' rcParam if you rely on a specific value.
A higher log-score (or a lower deviance) indicates a model with better predictive accuracy.
  "\nThe scale is now log by default. Use 'scale' argument or "


,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
pooled_model_pooled_data,0,-317.914,1.84283,0,0.913734,17.3918,0,False,log
partpooled_model_partpooled_data,1,-347.201,22.2082,29.2871,0.0862663,18.6456,22.8809,False,log
unpooled_model_unpooled_data,2,-405.831,10.4153,87.9171,6.36613e-15,18.641,22.2152,False,log
